In [13]:
import os 
import json
import random   
from typing import Annotated
from openai import AsyncOpenAI
from dotenv import load_dotenv
from IPython.display import display, HTML
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent
from semantic_kernel.functions import kernel_function
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, AzureChatCompletion

In [3]:
# Define a Plugin (Native Functions)
class DestinationsPlugin:
    def __init__(self):
        self.destinations = [
            "Barcelona, Spain", "Paris, France", "Berlin, Germany", "Tokyo, Japan", "Sydney, Australia",
            "New York, USA", "Cairo, Egypt", "Cape Town, South Africa", "Rio de Janeiro, Brazil", "Bali, Indonesia"
        ]
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)
        destination = random.choice(available_destinations)
        self.last_destination = destination
        return destination

Load Environment and Create OpenAI Client

In [4]:
load_dotenv(dotenv_path=".env", override=True)

endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
api_key  = os.environ["AZURE_OPENAI_API_KEY"]
deploy   = os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]
api_ver  = os.environ.get("AZURE_OPENAI_API_VERSION", "2025-01-01-preview")

# Giao tiếp trực tiếp với API GPT (qua OpenAI hoặc Azure).
client = AsyncOpenAI(
    api_key=api_key,
    base_url=f"{endpoint}/openai/deployments/{deploy}",
    default_headers={"api-key": api_key},
    default_query={"api-version": api_ver},
)

# Trung gian giữa Agent và client; wrap logic gửi prompt/nhận completion.
chat_completion_service = OpenAIChatCompletion(
    ai_model_id=deploy,         
    async_client=client,
)

# chat_completion_service = AzureChatCompletion(
#     deployment_name=deploy,
#     endpoint=endpoint,
#     api_key=api_key,
#     api_version=api_ver,
# )

In [5]:
print(endpoint)

https://ngothilinhau12915-ai-fo-resource.openai.azure.com


Creating the Agent

In [6]:
# Bộ não trung tâm: phân tích, đưa ra hành động, sử dụng plugin nếu cần.
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

Running the Agent

In [5]:
async def main():
    thread: ChatHistoryAgentThread | None = None
    try:
        user_inputs = ["Plan me a day trip."]
        for user_input in user_inputs:
            print(f"# User: {user_input}\n")
            first_chunk = True
            async for response in agent.invoke_stream(messages=user_input, thread=thread):
                if first_chunk:
                    who = response.name or "Agent"
                    print(f"# {who}: ", end="", flush=True)
                    first_chunk = False
                print(f"{response}", end="", flush=True)
                thread = response.thread
            print()
    finally:
        if thread:
            await thread.delete()

await main()

# User: Plan me a day trip.

# TravelAgent: How about a day trip to Berlin, Germany? Here’s a suggested itinerary:

### Morning
1. **Breakfast at a Local Café**: Start your day with a traditional German breakfast at a local café. Try some fresh bread rolls (Brötchen), cold cuts, and cheese.
2. **Brandenburg Gate**: Visit this iconic landmark and take some photos. It’s a symbol of Berlin’s history and unity.

### Late Morning
3. **Reichstag Building**: Head to the Reichstag, where you can take a tour of the building and enjoy the panoramic views from its glass dome. Be sure to register online in advance.

### Afternoon
4. **Lunch at a Street Food Market**: Enjoy lunch at one of Berlin’s famous street food markets, like Markthalle Neun, where you can find a variety of delicious options.
5. **Berlin Wall Memorial**: Visit the memorial to learn about the history of the Berlin Wall and see the preserved sections.

### Evening
6. **Explore the East Side Gallery**: Walk along the longest rema

In [7]:
AGENT_INSTRUCTIONS = """You are a helpful AI Agent that can help plan vacations for customers.

Important: When users specify a destination, always plan for that location. Only suggest random destinations when the user hasn't specified a preference.

When the conversation begins, introduce yourself with this message:
"Hello! I'm your TravelAgent assistant. I can help plan vacations and suggest interesting destinations for you. Here are some things you can ask me:
1. Plan a day trip to a specific location
2. Suggest a random vacation destination
3. Find destinations with specific features (beaches, mountains, historical sites, etc.)
4. Plan an alternative trip if you don't like my first suggestion

What kind of trip would you like me to help you plan today?"

Always prioritize user preferences. If they mention a specific destination like "Bali" or "Paris," focus your planning on that location rather than suggesting alternatives.
"""

agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions=AGENT_INSTRUCTIONS,
)

In [14]:
# Danh sách các prompt đầu vào mô phỏng cuộc trò chuyện qua nhiều lượt với agent.
user_inputs = [
    "Plan me a day trip.",
    "I don't like that destination. Plan me another vacation.",
]

async def main():
    thread: ChatHistoryAgentThread | None = None

    for user_input in user_inputs:
        html_output = (
            f"<div style='margin-bottom:10px'>"
            f"<div style='font-weight:bold'>User:</div>"
            f"<div style='margin-left:20px'>{user_input}</div></div>"
        )
        agent_name = None
        full_response: list[str] = []
        function_calls: list[str] = []
        current_function_name = None
        argument_buffer = ""

        # Gửi prompt vào agent qua invoke_stream()
        async for response in agent.invoke_stream(
            messages=user_input,
            thread=thread,
        ):
            thread = response.thread
            agent_name = response.name
            content_items = list(response.items)

            for item in content_items:
                if isinstance(item, FunctionCallContent):
                    if item.function_name:
                        current_function_name = item.function_name

                    # Accumulate arguments (streamed in chunks)
                    if isinstance(item.arguments, str):
                        argument_buffer += item.arguments
                elif isinstance(item, FunctionResultContent):
                    # Finalize any pending function call before showing result
                    if current_function_name:
                        formatted_args = argument_buffer.strip()
                        try:
                            parsed_args = json.loads(formatted_args)
                            formatted_args = json.dumps(parsed_args)
                        except Exception:
                            pass  # leave as raw string

                        function_calls.append(f"Calling function: {current_function_name}({formatted_args})")
                        current_function_name = None
                        argument_buffer = ""

                    function_calls.append(f"\nFunction Result:\n\n{item.result}")
                elif isinstance(item, StreamingTextContent) and item.text:
                    full_response.append(item.text)

        if function_calls:
            html_output += (
                "<div style='margin-bottom:10px'>"
                "<details>"
                "<summary style='cursor:pointer; font-weight:bold; color:#0066cc;'>Function Calls (click to expand)</summary>"
                "<div style='margin:10px; padding:10px; background-color:#f8f8f8; "
                "border:1px solid #ddd; border-radius:4px; white-space:pre-wrap; font-size:14px; color:#333;'>"
                f"{chr(10).join(function_calls)}"
                "</div></details></div>"
            )

        html_output += (
            "<div style='margin-bottom:20px'>"
            f"<div style='font-weight:bold'>{agent_name or 'Assistant'}:</div>"
            f"<div style='margin-left:20px; white-space:pre-wrap'>{''.join(full_response)}</div></div><hr>"
        )

        display(HTML(html_output))

await main()